# Practical Exam: Grocery Store Sales

FoodYum is a grocery store chain that is based in the United States.

Food Yum sells items such as produce, meat, dairy, baked goods, snacks, and other household food staples.

As food costs rise, FoodYum wants to make sure it keeps stocking products in all categories that cover a range of prices to ensure they have stock for a broad range of customers. 

## Data

The data is available in the table `products`.

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| product_type | Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). </br>Missing values should be replaced with “Unknown”. |
| brand | Nominal. The brand of the product. One of 7 possible values. </br>Missing values should be replaced with “Unknown”. |
| weight | Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median weight. |
| price | Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| average_units_sold | Discrete. The average number of units sold each month. This can be any positive integer value. </br>Missing values should be replaced with 0. |
| year_added | Nominal. The year the product was first added to FoodYum stock.</br>Missing values should be replaced with 2022. |
| stock_location | Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D </br>Missing values should be replaced with “Unknown”. |

# Task 1

Last year (2022) there was a bug in the product system. For some products that were added in that year, the `year_added` value was not set in the data. As the year the product was added may have an impact on the price of the product, this is important information to have. 

Write a query to determine how many products have the `year_added` value missing. Your output should be a single column, `missing_year`, with a single row giving the number of missing values.

In [34]:
-- Write your query for task 1 in this cell
SELECT COUNT(*) AS missing_year
FROM public.products
WHERE "year_added" IS NULL;

,missing_year
0,170


# Task 2

Given what you know about the year added data, you need to make sure all of the data is clean before you start your analysis. The table below shows what the data should look like. 

Write a query to ensure the product data matches the description provided. Do not update the original table.  

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| product_type | Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). </br>Missing values should be replaced with “Unknown”. |
| brand | Nominal. The brand of the product. One of 7 possible values. </br>Missing values should be replaced with “Unknown”. |
| weight | Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median weight. |
| price | Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| average_units_sold | Discrete. The average number of units sold each month. This can be any positive integer value. </br>Missing values should be replaced with 0. |
| year_added | Nominal. The year the product was first added to FoodYum stock.</br>Missing values should be replaced with last year (2022). |
| stock_location | Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D </br>Missing values should be replaced with “Unknown”. |

In [35]:
-- Write your query for task 2 in this cell
CREATE TEMP TABLE cleaning AS (
SELECT * FROM public.products);

--UPDATE MISSING VALUES ON brand
UPDATE cleaning
SET "brand"='Unknown'
WHERE TRIM("brand") ='_' OR TRIM("brand")='-';

--UPDATE MISSING VALUES ON weight + CASTING DATA TYPE
UPDATE cleaning
SET "weight"=TRIM("weight",' grams');
ALTER TABLE cleaning ALTER "weight" TYPE FLOAT USING "weight"::FLOAT;

--UPDATE MISSING VALUES ON YEAR ADDED
UPDATE cleaning
SET "year_added"=2022
WHERE "year_added" IS NULL;

--UPDATE VALUES IN LOCATION CATEGORY to UPPERCASE
UPDATE cleaning
SET "stock_location" = UPPER("stock_location");

--CAST TEMPLATE
--ALTER TABLE cleaning ALTER "Dine in option" TYPE bool USING "Dine in option"::boolean;
--ALTER TABLE cleaning ALTER "Takeout option" TYPE bool USING "Takeout option"::boolean;

--UPDATE TEMPLATE
--UPDATE cleaning
--SET "Reviews" = (SELECT (percentile_disc(0.5) WITHIN GROUP (ORDER BY "Reviews"))FROM cleaning)
--WHERE "Reviews" IS NULL;

SELECT *
FROM cleaning;


,product_id,product_type,brand,weight,price,average_units_sold,year_added,stock_location
0,1281,Meat,YumMie,489.91,11.95,25,2016,B
1,1583,Bakery,YumMie,537.37,6.73,16,2018,A
2,2,Produce,SilverLake,478.26,8.08,22,2022,C
3,3,Produce,TastyTreat,532.38,6.16,21,2018,B
4,4,Bakery,StandardYums,453.43,7.26,21,2021,D
...,...,...,...,...,...,...,...,...
1695,1661,Meat,SmoothTaste,509.69,14.24,26,2022,A
1696,1672,Produce,GoldTree,441.90,7.95,21,2022,D
1697,1673,Bakery,GoldTree,413.57,11.07,16,2022,A
1698,1686,Meat,GoldTree,658.60,16.07,25,2022,D


# Task 3

To find out how the range varies for each product type, your manager has asked you to determine the minimum and maximum values for each product type.   

Write a query to return the `product_type`, `min_price` and `max_price` columns. 

In [36]:
-- Write your query for task 3 in this cell
SELECT "product_type",
MIN("price") AS min_price,
MAX("price") AS max_price
FROM public.products
GROUP BY "product_type";

,product_type,min_price,max_price
0,Snacks,5.20,10.72
1,Produce,3.46,8.78
2,Dairy,8.33,13.97
3,Bakery,6.26,11.88
4,Meat,11.48,16.98


# Task 4

The team want to look in more detail at meat and dairy products where the average units sold was greater than ten. 

Write a query to return the `product_id`, `price` and `average_units_sold` of the rows of interest to the team. 

In [37]:
-- Write your query for task 4 in this cell
SELECT "product_id","price","average_units_sold"
FROM public.products
WHERE "average_units_sold">10
	AND "product_type" IN ('Meat','Dairy');

,product_id,price,average_units_sold
0,6,16.20,24
1,8,15.77,28
2,9,11.57,30
3,10,13.94,27
4,11,9.26,26
...,...,...,...
693,1694,16.00,25
694,1695,12.88,20
695,1696,14.08,25
696,1697,16.13,25
